In [ ]:
kmer_hits = '../out/associations/summary_cont_lmm_kmer.tsv'
names = '../out/associations/associated_ogs.final.tsv'
pangenome = '../out/roary/gene_presence_absence.csv'
ref_strain = 'IAI39'
ref_annotation_dir = '../out/annotations'
hpi = '../data/hpi.tsv'
outdir = '../out/plots/maps'

In [ ]:
outdir1 = outdir + '1'
outdir2 = outdir + '2'

In [ ]:
ref_genome = '%s/%s/%s.gbk' % (ref_annotation_dir, ref_strain, ref_strain)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors as cl

sns.set_style('white')

plt.rc('font', size=12)

In [ ]:
import os
import numpy as np
import pandas as pd
from Bio import SeqIO
from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram

In [ ]:
rd = {}
roary = pd.read_table(pangenome,
                      sep=',',
                      low_memory=False)
roary.set_index('Gene', inplace=True)
# Drop the other info columns
roary.drop(list(roary.columns[:13]), axis=1, inplace=True)
roary.reset_index(inplace=True)
for strain in roary.columns[1:]:
    for x, y, in roary.set_index(strain)['Gene'].dropna().to_dict().items():
        if str(x) == 'nan':
            continue
        # be aware of paralogs
        for g in x.split('\t'):
            rd[g] = y

In [ ]:
u = pd.read_table(names)
n = u.set_index('query')['preferred_og_name'].to_dict()
nsize = u.set_index('query')['representative_protein_length']
k = pd.read_table(kmer_hits,
                  index_col=0)
k['normalized_hits'] = (k['length'] / nsize)
ogs = set(k[k['normalized_hits'] >= 0.1].index)
cmap = plt.get_cmap('viridis')
cmap.set_under(sns.xkcd_rgb['light grey'])
norm = cl.Normalize(vmin=0.,vmax=k['normalized_hits'].max())
khits = k['normalized_hits'].to_dict()

In [ ]:
hpic = {x.split()[0]: c
        for x, c in zip(open(hpi),
                        sns.color_palette('Set2', len(open(hpi).readlines())))}
hpic['irp1'] = sns.color_palette('tab20', 2)[0]
hpic['irp3'] = sns.color_palette('tab20', 2)[1]
ogs = set(hpic.keys())

In [ ]:
seq = SeqIO.read(ref_genome,
                 'genbank')

In [ ]:
feats = [rd.get(f.qualifiers.get('locus_tag', [''])[0],
                f.qualifiers.get('locus_tag', [''])[0])
         for f in seq.features
         if f.type == 'gene' 
         and 'locus_tag' in f.qualifiers]
idxs = [feats.index(x) for x in ogs
        if x in feats]
ext_ogs = feats[min(idxs)-10:max(idxs)+10]

In [ ]:
img = 1

In [ ]:
gd = GenomeDiagram.Diagram('associated')
track = gd.new_track(1,
                     name='Reference strain (%s)' % ref_strain,
                     greytrack=True)
features = track.new_set()

pmin = None
pmax = None
for f in seq.features:
    if f.type != 'gene' or 'locus_tag' not in f.qualifiers:
        continue
    locus = f.qualifiers.get('locus_tag', [''])[0]
    og = rd.get(locus, locus)
    if og not in ext_ogs:
        continue
    gmin = int(f.location.start)
    gmax = int(f.location.end)
    if pmin is None or gmin < pmin:
        pmin = gmin
    if pmax is None or gmax > pmax:
        pmax = gmax
    if og == 'pks2':
        if gmax - gmin < 2000:
            name = 'irp3'
            og  = 'irp3'
        else:
            name = 'irp1'
            og = 'irp1'
    else:
        name = n.get(og, og)
    features.add_feature(f,
                         label=True,
                         name=name,
                         color=hpic.get(og, sns.xkcd_rgb['light grey']),
                         label_size=5,
                         label_angle=90,
                         label_position='middle',
                         sigil='ARROW',
                         arrowshaft_height=0.5)
gd.draw(format='linear',
        fragments=1,
        orientation='landscape',
        start=pmin, end=pmax,
        pagesize=(((pmax - pmin)/1000)*cm, 3*cm))
gd.write(os.path.join(outdir, '%03d_ref_chromosome.svg' % img), 'SVG')

In [ ]:
for strain in sorted(set(roary.columns[1:]).difference([ref_strain,])):
    img += 1
    for seq in SeqIO.parse('%s/%s/%s.gbk' % (ref_annotation_dir,
                                             strain, strain),
                           'genbank'):
        feats = [rd.get(f.qualifiers['locus_tag'][0],
                        f.qualifiers['locus_tag'][0])
                 for f in seq.features
                 if f.type == 'gene'
                 and 'locus_tag' in f.qualifiers]
        if len(set(feats).intersection(ogs)) == 0:
            continue
        idxs = [feats.index(x) for x in ogs
                if x in feats]
        ext_feats = feats[min(idxs)-10 if min(idxs)-10 > 10 else 0:max(idxs)+10]
        gd = GenomeDiagram.Diagram('test')
        track = gd.new_track(1,
                             name=strain,
                             greytrack=True)
        features = track.new_set()

        pmin = None
        pmax = None
        for f in seq.features:
            if f.type != 'gene' or 'locus_tag' not in f.qualifiers:
                continue
            locus = f.qualifiers.get('locus_tag', [''])[0]
            og = rd.get(locus, locus)
            if og not in ext_feats:
                continue
            gmin = int(f.location.start)
            gmax = int(f.location.end)
            if pmin is None or gmin < pmin:
                pmin = gmin
            if pmax is None or gmax > pmax:
                pmax = gmax
            if og == 'pks2':
                if gmax - gmin < 2000:
                    name = 'irp3'
                    og  = 'irp3'
                else:
                    name = 'irp1'
                    og = 'irp1'
            else:
                name = n.get(og, og)
            features.add_feature(f,
                                 label=True,
                                 name=name,
                                 color=hpic.get(og, sns.xkcd_rgb['light grey']),
                                 label_size=5,
                                 label_angle=90,
                                 label_position='middle',
                                 sigil='ARROW',
                                 arrowshaft_height=0.5)
        gd.draw(format='linear',
                fragments=1,
                orientation='landscape',
                start=pmin, end=pmax,
                pagesize=(((pmax - pmin)/1000)*cm, 3*cm))
        gd.write(os.path.join(outdir, '%03d_%s_%s.svg' % (img, strain, seq.id)), 'SVG')

In [ ]:
files = [x for x in os.listdir(outdir)
         if 'final' not in x
         and x.endswith('.svg')]
for img in sorted({x.split('_')[0] for x in files}):
    gfiles = [x for x in files
              if x.startswith(img)]
    os.system('python3 ../src/svg_stack.py %s/%s_* --margin 5px --direction h > %s/%sfinal.svg' % (outdir, img,
                                                                                                   outdir, img))

In [ ]:
os.system('python3 ../src/svg_stack.py %s/*final.svg --margin 5px --direction v > %s/all.svg' % (outdir,
                                                                                                 outdir))

In [ ]:
ogs = {'iutA', 'iucD', 'iucC'}

In [ ]:
hpic = {x.split()[0]: c
        for x, c in zip(ogs,
                        sns.color_palette('Set2', len(ogs)))}

In [ ]:
seq = SeqIO.read(ref_genome,
                 'genbank')

In [ ]:
feats = [rd.get(f.qualifiers.get('locus_tag', [''])[0],
                f.qualifiers.get('locus_tag', [''])[0])
         for f in seq.features
         if f.type == 'gene' 
         and 'locus_tag' in f.qualifiers]
idxs = [feats.index(x) for x in ogs
        if x in feats]
ext_ogs = feats[min(idxs)-10:max(idxs)+10]

In [ ]:
img = 1

In [ ]:
gd = GenomeDiagram.Diagram('associated')
track = gd.new_track(1,
                     name='Reference strain (%s)' % ref_strain,
                     greytrack=True)
features = track.new_set()

pmin = None
pmax = None
for f in seq.features:
    if f.type != 'gene' or 'locus_tag' not in f.qualifiers:
        continue
    locus = f.qualifiers.get('locus_tag', [''])[0]
    og = rd.get(locus, locus)
    if og not in ext_ogs:
        continue
    gmin = int(f.location.start)
    gmax = int(f.location.end)
    if pmin is None or gmin < pmin:
        pmin = gmin
    if pmax is None or gmax > pmax:
        pmax = gmax
    if og == 'pks2':
        if gmax - gmin < 2000:
            name = 'irp3'
            og  = 'irp3'
        else:
            name = 'irp1'
            og = 'irp1'
    else:
        name = n.get(og, og)
    features.add_feature(f,
                         label=True,
                         name=name,
                         color=hpic.get(og, sns.xkcd_rgb['light grey']),
                         label_size=5,
                         label_angle=90,
                         label_position='middle',
                         sigil='ARROW',
                         arrowshaft_height=0.5)
gd.draw(format='linear',
        fragments=1,
        orientation='landscape',
        start=pmin, end=pmax,
        pagesize=(((pmax - pmin)/1000)*cm, 3*cm))
gd.write(os.path.join(outdir1, '%03d_ref_chromosome.svg' % img), 'SVG')

In [ ]:
for strain in sorted(set(roary.columns[1:]).difference([ref_strain,])):
    img += 1
    for seq in SeqIO.parse('%s/%s/%s.gbk' % (ref_annotation_dir,
                                             strain, strain),
                           'genbank'):
        feats = [rd.get(f.qualifiers['locus_tag'][0],
                        f.qualifiers['locus_tag'][0])
                 for f in seq.features
                 if f.type == 'gene'
                 and 'locus_tag' in f.qualifiers]
        if len(set(feats).intersection(ogs)) == 0:
            continue
        idxs = [feats.index(x) for x in ogs
                if x in feats]
        ext_feats = feats[min(idxs)-10 if min(idxs)-10 > 10 else 0:max(idxs)+10]
        gd = GenomeDiagram.Diagram('test')
        track = gd.new_track(1,
                             name=strain,
                             greytrack=True)
        features = track.new_set()

        pmin = None
        pmax = None
        for f in seq.features:
            if f.type != 'gene' or 'locus_tag' not in f.qualifiers:
                continue
            locus = f.qualifiers.get('locus_tag', [''])[0]
            og = rd.get(locus, locus)
            if og not in ext_feats:
                continue
            gmin = int(f.location.start)
            gmax = int(f.location.end)
            if pmin is None or gmin < pmin:
                pmin = gmin
            if pmax is None or gmax > pmax:
                pmax = gmax
            if og == 'pks2':
                if gmax - gmin < 2000:
                    name = 'irp3'
                    og  = 'irp3'
                else:
                    name = 'irp1'
                    og = 'irp1'
            else:
                name = n.get(og, og)
            features.add_feature(f,
                                 label=True,
                                 name=name,
                                 color=hpic.get(og, sns.xkcd_rgb['light grey']),
                                 label_size=5,
                                 label_angle=90,
                                 label_position='middle',
                                 sigil='ARROW',
                                 arrowshaft_height=0.5)
        gd.draw(format='linear',
                fragments=1,
                orientation='landscape',
                start=pmin, end=pmax,
                pagesize=(((pmax - pmin)/1000)*cm, 3*cm))
        gd.write(os.path.join(outdir1, '%03d_%s_%s.svg' % (img, strain, seq.id)), 'SVG')

In [ ]:
files = [x for x in os.listdir(outdir1)
         if 'final' not in x
         and x.endswith('.svg')]
for img in sorted({x.split('_')[0] for x in files}):
    gfiles = [x for x in files
              if x.startswith(img)]
    os.system('python3 ../src/svg_stack.py %s/%s_* --margin 5px --direction h > %s/%sfinal.svg' % (outdir1, img,
                                                                                                   outdir1, img))

In [ ]:
os.system('python3 ../src/svg_stack.py %s/*final.svg --margin 5px --direction v > %s/all.svg' % (outdir1,
                                                                                                 outdir1))

In [ ]:
ogs = {'mntB', 'mntB_2', 'btuD_2', 'hpf'}

In [ ]:
hpic = {x.split()[0]: c
        for x, c in zip(ogs,
                        sns.color_palette('Set2', len(ogs)))}

In [ ]:
seq = SeqIO.read(ref_genome,
                 'genbank')

In [ ]:
feats = [rd.get(f.qualifiers.get('locus_tag', [''])[0],
                f.qualifiers.get('locus_tag', [''])[0])
         for f in seq.features
         if f.type == 'gene' 
         and 'locus_tag' in f.qualifiers]
idxs = [feats.index(x) for x in ogs
        if x in feats]
ext_ogs = feats[min(idxs)-10:max(idxs)+10]

In [ ]:
img = 1

In [ ]:
gd = GenomeDiagram.Diagram('associated')
track = gd.new_track(1,
                     name='Reference strain (%s)' % ref_strain,
                     greytrack=True)
features = track.new_set()

pmin = None
pmax = None
for f in seq.features:
    if f.type != 'gene' or 'locus_tag' not in f.qualifiers:
        continue
    locus = f.qualifiers.get('locus_tag', [''])[0]
    og = rd.get(locus, locus)
    if og not in ext_ogs:
        continue
    gmin = int(f.location.start)
    gmax = int(f.location.end)
    if pmin is None or gmin < pmin:
        pmin = gmin
    if pmax is None or gmax > pmax:
        pmax = gmax
    if og == 'pks2':
        if gmax - gmin < 2000:
            name = 'irp3'
            og  = 'irp3'
        else:
            name = 'irp1'
            og = 'irp1'
    else:
        name = n.get(og, og)
    features.add_feature(f,
                         label=True,
                         name=name,
                         color=hpic.get(og, sns.xkcd_rgb['light grey']),
                         label_size=5,
                         label_angle=90,
                         label_position='middle',
                         sigil='ARROW',
                         arrowshaft_height=0.5)
gd.draw(format='linear',
        fragments=1,
        orientation='landscape',
        start=pmin, end=pmax,
        pagesize=(((pmax - pmin)/1000)*cm, 3*cm))
gd.write(os.path.join(outdir2, '%03d_ref_chromosome.svg' % img), 'SVG')

In [ ]:
for strain in sorted(set(roary.columns[1:]).difference([ref_strain,])):
    img += 1
    for seq in SeqIO.parse('%s/%s/%s.gbk' % (ref_annotation_dir,
                                             strain, strain),
                           'genbank'):
        feats = [rd.get(f.qualifiers['locus_tag'][0],
                        f.qualifiers['locus_tag'][0])
                 for f in seq.features
                 if f.type == 'gene'
                 and 'locus_tag' in f.qualifiers]
        if len(set(feats).intersection(ogs)) == 0:
            continue
        idxs = [feats.index(x) for x in ogs
                if x in feats]
        ext_feats = feats[min(idxs)-10 if min(idxs)-10 > 10 else 0:max(idxs)+10]
        gd = GenomeDiagram.Diagram('test')
        track = gd.new_track(1,
                             name=strain,
                             greytrack=True)
        features = track.new_set()

        pmin = None
        pmax = None
        for f in seq.features:
            if f.type != 'gene' or 'locus_tag' not in f.qualifiers:
                continue
            locus = f.qualifiers.get('locus_tag', [''])[0]
            og = rd.get(locus, locus)
            if og not in ext_feats:
                continue
            gmin = int(f.location.start)
            gmax = int(f.location.end)
            if pmin is None or gmin < pmin:
                pmin = gmin
            if pmax is None or gmax > pmax:
                pmax = gmax
            if og == 'pks2':
                if gmax - gmin < 2000:
                    name = 'irp3'
                    og  = 'irp3'
                else:
                    name = 'irp1'
                    og = 'irp1'
            else:
                name = n.get(og, og)
            features.add_feature(f,
                                 label=True,
                                 name=name,
                                 color=hpic.get(og, sns.xkcd_rgb['light grey']),
                                 label_size=5,
                                 label_angle=90,
                                 label_position='middle',
                                 sigil='ARROW',
                                 arrowshaft_height=0.5)
        gd.draw(format='linear',
                fragments=1,
                orientation='landscape',
                start=pmin, end=pmax,
                pagesize=(((pmax - pmin)/1000)*cm, 3*cm))
        gd.write(os.path.join(outdir2, '%03d_%s_%s.svg' % (img, strain, seq.id)), 'SVG')

In [ ]:
files = [x for x in os.listdir(outdir2)
         if 'final' not in x
         and x.endswith('.svg')]
for img in sorted({x.split('_')[0] for x in files}):
    gfiles = [x for x in files
              if x.startswith(img)]
    os.system('python3 ../src/svg_stack.py %s/%s_* --margin 5px --direction h > %s/%sfinal.svg' % (outdir2, img,
                                                                                                   outdir2, img))

In [ ]:
os.system('python3 ../src/svg_stack.py %s/*final.svg --margin 5px --direction v > %s/all.svg' % (outdir2,
                                                                                                 outdir2))